In [ ]:
!gdown --id '1qWFlioM9B_R83sn4GfWPltki3kpsZHxo' --output 'dataset.zip'
!unzip dataset.zip -d '/content/data'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1qWFlioM9B_R83sn4GfWPltki3kpsZHxo
To: /content/dataset.zip
100% 981k/981k [00:00<00:00, 161MB/s]
Archive:  dataset.zip
  inflating: /content/data/resized/3/162.jpg  
  inflating: /content/data/resized/3/1112.jpg  
  inflating: /content/data/resized/3/620.jpg  
  inflating: /content/data/resized/3/1201.jpg  
  inflating: /content/data/resized/3/152.jpg  
  inflating: /content/data/resized/3/612.jpg  
  inflating: /content/data/resized/3/1251.jpg  
  inflating: /content/data/resized/3/691.jpg  
  inflating: /content/data/resized/3/372.jpg  
  inflating: /content/data/resized/3/1451.jpg  
  inflating: /content/data/resized/3/941.jpg  
  inflating: /content/data/resized/3/1.jpg  
  inflating: /content/data/resized/3/90

In [ ]:
!gdown --id '1fOdMVmWvLNZXVKDQgU8sBS3RR_E8FTC0' --output 'dataset.zip'
!unzip dataset.zip -d '/content/data'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1fOdMVmWvLNZXVKDQgU8sBS3RR_E8FTC0
To: /content/dataset.zip
100% 148k/148k [00:00<00:00, 103MB/s]
Archive:  dataset.zip
   creating: /content/data/resized_test/
   creating: /content/data/resized_test/0/
  inflating: /content/data/resized_test/0/1619.jpg  
  inflating: /content/data/resized_test/0/1620.jpg  
  inflating: /content/data/resized_test/0/1621.jpg  
  inflating: /content/data/resized_test/0/1623.jpg  
  inflating: /content/data/resized_test/0/1624.jpg  
  inflating: /content/data/resized_test/0/1625.jpg  
  inflating: /content/data/resized_test/0/1626.jpg  
  inflating: /content/data/resized_test/0/1627.jpg  
  inflating: /content/data/resized_test/0/1628.jpg  
  inflating: /content/data/resized_test/0/16

In [ ]:
import tensorflow as tf
import os
import pathlib
import numpy as np

In [ ]:
data_dir = pathlib.Path('/content/data/resized')

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

dst = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=True)
dst = dst.shuffle(image_count, reshuffle_each_iteration=True)

val_size = int(image_count * 0.2)
train_set = dst.skip(val_size)
val_set = dst.take(val_size)

182


In [ ]:
class_names = ['0', '1', '2', '3', '4']
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  # return tf.argmax(one_hot)
  return one_hot

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(img, channels=1)
  #img = tf.keras.layers.Rescaling(1./255)(img)
  img = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)(img)
  return img, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_set = train_set.map(process_path, num_parallel_calls=AUTOTUNE)
val_set = val_set.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in train_set.take(10):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (32, 640, 1)
Label:  [False False False  True False]
Image shape:  (32, 640, 1)
Label:  [False False  True False False]
Image shape:  (32, 640, 1)
Label:  [False False  True False False]
Image shape:  (32, 640, 1)
Label:  [False False False  True False]
Image shape:  (32, 640, 1)
Label:  [False False  True False False]
Image shape:  (32, 640, 1)
Label:  [False False False  True False]
Image shape:  (32, 640, 1)
Label:  [ True False False False False]
Image shape:  (32, 640, 1)
Label:  [False False False  True False]
Image shape:  (32, 640, 1)
Label:  [ True False False False False]
Image shape:  (32, 640, 1)
Label:  [False  True False False False]


In [ ]:
batch_size = 8

def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_set = configure_for_performance(train_set)
val_set = configure_for_performance(val_set)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense
from tensorflow.keras.layers import Activation, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [ ]:
num_classes = 5

# Model create #1
filter_x = 3;
filter_y = 3;

input_shape = (32, 640, 1)

with tf.device('/device:GPU:0'):
  model=Sequential()
  #Conv1
  model.add(Conv2D(filters=16, 
          kernel_size=(filter_x, filter_y), 
          padding="same",  
          input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D()) # 16 320 16

  #Conv2
  model.add(Conv2D(filters=16, 
          kernel_size=(filter_x, filter_y), 
          padding="same", 
          input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D()) # 8 160 16

  #Conv3
  model.add(Conv2D(filters=16, 
          kernel_size=(filter_x, filter_y), 
          padding="same", 
          input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D()) # 4 80 32

  #Conv4
  model.add(Conv2D(filters=32, 
          kernel_size=(filter_x, filter_y), 
          padding="same", 
          input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(MaxPooling2D()) # 2 40 64

  # Model create #2(continue)
  #FC1
  model.add(Conv2D(filters=32, 
          kernel_size=(2, 1), 
          padding="valid", 
          input_shape=input_shape)) # 1 40 64
  model.add(Conv2D(filters=5, 
          kernel_size=(1, 1), 
          padding="valid", 
          input_shape=input_shape)) # 1 40 5
  model.add(Activation("softmax"))
  model.add(Flatten()) #也許要修正這裡
  model.add(Dense(64))
  model.add(Dense(20))
  model.add(Dense(20))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  #FC2
  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  
  # Training model
  #Define optimizer loss function and merics 
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  # Set training
  model.fit(train_set, validation_data=val_set, epochs=200)

Epoch 1/200
19/19 [==============================] - 2s 35ms/step - loss: 1.5367 - accuracy: 0.3630 - val_loss: 1.6115 - val_accuracy: 0.2500
Epoch 2/200
19/19 [==============================] - 0s 17ms/step - loss: 1.2580 - accuracy: 0.5479 - val_loss: 1.6401 - val_accuracy: 0.2500
Epoch 3/200
19/19 [==============================] - 0s 17ms/step - loss: 1.1000 - accuracy: 0.5685 - val_loss: 1.6952 - val_accuracy: 0.2500
Epoch 4/200
19/19 [==============================] - 0s 16ms/step - loss: 0.9418 - accuracy: 0.6986 - val_loss: 1.7432 - val_accuracy: 0.2500
Epoch 5/200
19/19 [==============================] - 0s 19ms/step - loss: 0.8740 - accuracy: 0.7192 - val_loss: 1.7694 - val_accuracy: 0.2500
Epoch 6/200
19/19 [==============================] - 0s 16ms/step - loss: 0.7842 - accuracy: 0.7466 - val_loss: 1.8493 - val_accuracy: 0.2500
Epoch 7/200
19/19 [==============================] - 0s 19ms/step - loss: 0.7821 - accuracy: 0.7466 - val_loss: 2.0023 - val_accuracy: 0.2500
Epoch 

In [ ]:
import cv2
test_img = cv2.imread('/content/data/resized/2/791.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
test_img = np.expand_dims(test_img, axis=0)
test_img = np.expand_dims(test_img, axis=3)
print(test_img.shape)
print(model(test_img))

(1, 32, 640, 1)
tf.Tensor([[2.3048591e-08 9.9999976e-01 7.6755626e-08 1.5335531e-11 1.7805809e-07]], shape=(1, 5), dtype=float32)


In [ ]:
def process_path2(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(img, channels=1)
  img = tf.cast(img, tf.float32)
  #img = tf.keras.layers.Rescaling(1./255)(img)
  return img, label

dst = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
AUTOTUNE = tf.data.AUTOTUNE

dst = dst.map(process_path, num_parallel_calls=AUTOTUNE)
dst = dst.batch(1)

for image, label in dst.take(10):
  '''
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  '''
  print([image])

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
'''
files = pathlib.Path("/content/data/resized").glob('**/*.jpg')
def representative_data_gen():
  for file in files:
    test_img = cv2.imread(str(file))
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    test_img = np.expand_dims(test_img, axis=0)
    test_img = np.expand_dims(test_img, axis=3)
    test_img = np.float32(test_img)
    yield [test_img]
'''
def representative_data_gen():
  for input_value, label in dst.take(100):
    yield [input_value]
'''
from pathlib import Path
files = Path("/content/data/resized").glob('**/*.jpg')
for file in files:
  test_img = cv2.imread(str(file))
  test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
  test_img = np.expand_dims(test_img, axis=0)
  print(test_img.shape)
'''
converter.representative_dataset = representative_data_gen
converted_model = converter.convert()
generated_dir = pathlib.Path("/content/")
generated_dir.mkdir(exist_ok=True,parents=True)
converted_model_file = generated_dir/"model_int8.tflite"
converted_model_file.write_bytes(converted_model)

[<tf.Tensor: shape=(1, 32, 640, 1), dtype=float32, numpy=
array([[[[-0.20784307],
         [-0.20784307],
         [-0.20784307],
         ...,
         [-0.15294111],
         [-0.15294111],
         [-0.16862738]],

        [[-0.20784307],
         [-0.2156862 ],
         [-0.2156862 ],
         ...,
         [-0.15294111],
         [-0.15294111],
         [-0.16862738]],

        [[-0.20784307],
         [-0.20784307],
         [-0.2156862 ],
         ...,
         [-0.15294111],
         [-0.15294111],
         [-0.16862738]],

        ...,

        [[-0.23137248],
         [-0.23137248],
         [-0.23137248],
         ...,
         [-0.15294111],
         [-0.15294111],
         [-0.15294111]],

        [[-0.23137248],
         [-0.23137248],
         [-0.23137248],
         ...,
         [-0.15294111],
         [-0.15294111],
         [-0.15294111]],

        [[-0.23137248],
         [-0.23137248],
         [-0.23137248],
         ...,
         [-0.15294111],
         [-0.15294

INFO:tensorflow:Assets written to: /tmp/tmp951yi_fl/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


38128

In [ ]:
import cv2
interpreter = tf.lite.Interpreter(model_path=str(converted_model_file))
#print(interpreter.get_input_details())
interpreter.allocate_tensors()
#print(interpreter.get_input_details())
test_data_dir = pathlib.Path('/content/data/resized')
# test_data_dir = pathlib.Path('/content/data/resized_test')

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
scale, zero_point = interpreter.get_output_details()[0]['quantization']

tflm_correct = 0
model_correct = 0
total_test = 0
for f in test_data_dir.glob('*/*.jpg'):
  img = cv2.imread(str(f))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = np.expand_dims(img, axis=0)
  img = np.expand_dims(img, axis=3)
  img = np.float32(img)
  img = img/127.5 -1
  model_result = np.argmax(model(img))
  
  img = np.int8(img / scale + zero_point)
  # print(img.shape)
  interpreter.set_tensor(input_index, img)

  interpreter.invoke()

  # Find the letter with highest probability
  output = interpreter.tensor(output_index)
  #result = np.array(output()[0])
  result = np.argmax(output()[0])
  print(str(f)+', tflm_result='+str(result)+', model_result='+str(model_result))
  s = str(f).split('/')
  if(int(s[-2]) == result):
    tflm_correct = tflm_correct+1
  if(int(s[-2]) == model_result):
    model_correct = model_correct+1
  total_test = total_test+1

/content/data/resized/3/1060.jpg, tflm_result=2, model_result=3
/content/data/resized/3/1280.jpg, tflm_result=1, model_result=3
/content/data/resized/3/372.jpg, tflm_result=1, model_result=3
/content/data/resized/3/383.jpg, tflm_result=1, model_result=3
/content/data/resized/3/500.jpg, tflm_result=1, model_result=3
/content/data/resized/3/162.jpg, tflm_result=1, model_result=3
/content/data/resized/3/1112.jpg, tflm_result=3, model_result=3
/content/data/resized/3/241.jpg, tflm_result=3, model_result=3
/content/data/resized/3/941.jpg, tflm_result=3, model_result=3
/content/data/resized/3/1120.jpg, tflm_result=0, model_result=3
/content/data/resized/3/121.jpg, tflm_result=1, model_result=3
/content/data/resized/3/1251.jpg, tflm_result=3, model_result=3
/content/data/resized/3/1201.jpg, tflm_result=1, model_result=3
/content/data/resized/3/51.jpg, tflm_result=1, model_result=3
/content/data/resized/3/142.jpg, tflm_result=1, model_result=3
/content/data/resized/3/133.jpg, tflm_result=1, mo

In [ ]:
print('tflm accuracy = '+str(tflm_correct/total_test*100)+'%')
print('model accuracy = '+str(model_correct/total_test*100)+'%')

tflm accuracy = 51.64835164835166%
model accuracy = 92.3076923076923%


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 32, 640, 16)       160       
                                                                 
 batch_normalization_25 (Bat  (None, 32, 640, 16)      64        
 chNormalization)                                                
                                                                 
 activation_35 (Activation)  (None, 32, 640, 16)       0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 16, 320, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 16, 320, 16)       2320      
                                                                 
 batch_normalization_26 (Bat  (None, 16, 320, 16)     